In [ ]:
!tar zxvf ./drive/My\ Drive/Colab\ Notebooks/train.tgz
!pip install wandb

In [ ]:
!wandb login

# CAPTCHA Recognition - 6 digits

In [1]:
import tensorflow as tf
import os
import numpy as np
import pandas as pd

In [2]:
from tensorflow.keras import backend
from tensorflow.keras import layers

In [3]:
import wandb

# Parsing Data

In [4]:
CLASS_NAMES = np.array([])
for i in range(10):
    CLASS_NAMES =np.append(CLASS_NAMES,chr(ord("0")+i))
for i in range(26):
    CLASS_NAMES = np.append(CLASS_NAMES,chr(ord("A")+i))

#functions
def parse_label(strIN):
    return (strIN==CLASS_NAMES).astype(float)
def readimg_to_tensor(fn):
    a = tf.io.read_file((fn))
    img = tf.io.decode_jpeg(a)
    return img

In [5]:
#training data 01 
train_dir_data01 = os.path.abspath(os.getcwd())+"/train/data01_train/"
data = pd.read_csv("./train/data01_train.csv")
for i in range(len(data)):
    data.iloc[i,1] = list(data.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data)):
        arr[j][i] = data.iloc[i,1][j]
data = data.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

for i in range(6):
    data.iloc[:,i+1] = data.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data.iloc[:,6].values))

train_dir = tf.data.Dataset.list_files(train_dir_data01+'*.jpg',shuffle=False) 
train_data = train_dir.map(lambda x: readimg_to_tensor(x))
train_label = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data01=tf.data.Dataset.zip((train_data,train_label))

In [ ]:
#training data 02
train_dir_data02 = os.path.abspath(os.getcwd())+"/train/data02_train/"
data2 = pd.read_csv("./train/data02_train.csv")
for i in range(len(data2)):
    data2.iloc[i,1] = list(data2.iloc[i,1])
arr = np.zeros([6,50000],str)
for j in range(6):
    for i in range(len(data2)):
        arr[j][i] = data2.iloc[i,1][j]
data2 = data2.drop(columns= ["code"]).join(pd.DataFrame(arr.transpose(),columns = ["code0","code1",'code2','code3','code4','code5']))

#to one hot
for i in range(6):
    data2.iloc[:,i+1] = data2.iloc[:,i+1].map(parse_label)

TL = tf.convert_to_tensor(list(data2.iloc[:,1].values))
TL2 = tf.convert_to_tensor(list(data2.iloc[:,2].values))
TL3 = tf.convert_to_tensor(list(data2.iloc[:,3].values))
TL4 = tf.convert_to_tensor(list(data2.iloc[:,4].values))
TL5 = tf.convert_to_tensor(list(data2.iloc[:,5].values))
TL6 = tf.convert_to_tensor(list(data2.iloc[:,6].values))

train_dir2 = tf.data.Dataset.list_files(train_dir_data02+'*.jpg',shuffle=False) 
train_data2 = train_dir2.map(lambda x: readimg_to_tensor(x))
train_label2 = tf.data.Dataset.from_tensor_slices((TL,TL2,TL3,TL4,TL5,TL6))
train_data02=tf.data.Dataset.zip((train_data2,train_label2))

In [6]:
#number of element
num_elements = tf.data.experimental.cardinality(train_data01).numpy()
num_elements

50000

In [7]:
def prepData01(): 
  validation_split=0.1
  split = int(num_elements*validation_split)
  train_data_gen=iter(train_data01.shuffle(10000).batch(10000))

  img , lb = next(train_data_gen)
  #img_train = img[:-split]
  #lb_train = (lb[0][:-split],lb[1][:-split],lb[2][:-split],lb[3][:-split],lb[4][:-split],lb[5][:-split])
  #img_test = img[-split:]
  #lb_test = (lb[0][-split:],lb[1][-split:],lb[2][-split:],lb[3][-split:],lb[4][-split:],lb[5][-split:])
  return img,lb

# Model configeration

In [9]:
#ini and regul
initializer = tf.keras.initializers.he_normal( seed = 3)
alpha = 0.0001  # weight decay coefficient 0.001 Fail
regularizer = tf.keras.regularizers.l2(alpha)

In [18]:
#pre activate
def block2(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
  """A residual block.
  Arguments:
      x: input tensor.
      filters: integer, filters of the bottleneck layer.
      kernel_size: default 3, kernel size of the bottleneck layer.
      stride: default 1, stride of the first layer.
      conv_shortcut: default False, use convolution shortcut if True,
        otherwise identity shortcut.
      name: string, block label.
  Returns:
    Output tensor for the residual block.
  """
  bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

  preact = layers.BatchNormalization(
      axis=bn_axis, epsilon=1.001e-5, name=name + '_preact_bn')(x)
  preact = layers.Activation('relu', name=name + '_preact_relu')(preact)

  if conv_shortcut:
    shortcut = layers.Conv2D(
        4 * filters, 1, strides=stride, name=name + '_0_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  else:
    shortcut = layers.MaxPooling2D(1, strides=stride)(x) if stride > 1 else x

  x = layers.Conv2D(
      filters, 1, strides=1, use_bias=False, name=name + '_1_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(preact)
  x = layers.BatchNormalization(
      axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x)
  x = layers.Activation('relu', name=name + '_1_relu')(x)

  x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name=name + '_2_pad')(x)
  x = layers.Conv2D(
      filters,
      kernel_size,
      strides=stride,
      use_bias=False,
      name=name + '_2_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  x = layers.BatchNormalization(
      axis=bn_axis, epsilon=1.001e-5, name=name + '_2_bn')(x)
  x = layers.Activation('relu', name=name + '_2_relu')(x)

  x = layers.Conv2D(4 * filters, 1, name=name + '_3_conv', kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  x = layers.Dropout(0.3)(x)
  x = layers.Add(name=name + '_out')([shortcut, x])
  return x

def stack1(x, filters, blocks, stride1=2, name=None):
  """A set of stacked residual blocks.
  Arguments:
    x: input tensor.
    filters: integer, filters of the bottleneck layer in a block.
    blocks: integer, blocks in the stacked blocks.
    stride1: default 2, stride of the first layer in the first block.
    name: string, stack label.
  Returns:
    Output tensor for the stacked blocks.
  """
  x = block2(x, filters, conv_shortcut=True ,stride=stride1, name=name + '_block1')
  for i in range(2, blocks + 1):
    x = block2(x, filters, conv_shortcut=False, name=name + '_block' + str(i))
  return x

In [19]:
def buildModel(modelname="myResNet"):
  bn_axis = 3 
  #ResNet+RNN config
  def stack_fn(x):
    x = stack1(x, 64, 3, stride1=1, name='conv2')
    x = stack1(x, 128, 4, name='conv3')
    return stack1(x, 256, 3, name='conv4')
  #input shape
  img_input = layers.Input(shape=(60,200,3))

  x = layers.ZeroPadding2D(
      padding=((1, 1), (1, 1)), name='conv1_pad')(img_input)
  x = layers.Conv2D(32, 3, strides=2, use_bias=True, name='conv1_conv',kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  #pre act
  x = layers.ZeroPadding2D(padding=((1, 1), (1, 1)), name='pool1_pad')(x)
  x = layers.MaxPooling2D(3, strides=2, name='pool1_pool')(x)
  x = layers.Dropout(0.3)(x)
  x = stack_fn(x)
  
  x = layers.BatchNormalization(
        axis=bn_axis, epsilon=1.001e-5, name='post_bn')(x)
  x = layers.Activation('relu', name='post_relu')(x)
  x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
  #RNN
  x = layers.RepeatVector(6)(x)
  x = layers.GRU(128,return_sequences=True)(x)
  x = layers.GRU(128,return_sequences=True)(x)
  #classify
  digit1 = layers.Dense(36, name='digit1', activation='softmax',kernel_initializer = initializer,kernel_regularizer = regularizer)(x)
  #digit2 = layers.Dense(36, name='digit2', activation='softmax')(x)
  #digit3 = layers.Dense(36, name='digit3', activation='softmax')(x)
  #digit4 = layers.Dense(36, name='digit4', activation='softmax')(x)
  #digit5 = layers.Dense(36, name='digit5', activation='softmax')(x)
  #digit6 = layers.Dense(36, name='digit6', activation='softmax')(x)
  modelt = tf.keras.models.Model(inputs=img_input,outputs = digit1, name = modelname)
  modelt.compile(
    loss=[
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
        tf.keras.losses.CategoricalCrossentropy(),
    ],
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9), 
    metrics=['accuracy'])
  return modelt

In [20]:
mymodel = buildModel()

In [21]:
mymodel.summary()

Model: "myResNet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 60, 200, 3)] 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 62, 202, 3)   0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 30, 100, 32)  896         conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 32, 102, 32)  0           conv1_conv[0][0]                 
___________________________________________________________________________________________

In [14]:
img , lb= prepData01()

In [15]:
lb = tf.concat([tf.expand_dims(lbs,1) for lbs in lb],1)

In [27]:
def trainModel(model,img_train,lb_train,callB):
  history = model.fit(
        img_train,
        lb_train,
        batch_size = 32,
        shuffle = True,
        validation_split=0.1,
        use_multiprocessing=True,
        epochs=100,
        verbose=1,
        callbacks = callB
        )

In [22]:
config = {
    "stack" : "32+ 64,128,256 3,4,3blocks",
    "weight_regu" : 0.0001,
    "dropout" : 0.3,
    "preact" : True,
    "name" : "ResNet_RNN",
    "output_WR" : 1,
    "total_params" : 5741604
}

In [23]:
wandb.init(project="my-project-2",reinit=True,name=config["name"], config=config)

Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable


W&B Run: https://app.wandb.ai/tingww/my-project-2/runs/39shh7kx

Run pip install nbformat to save notebook history


In [29]:
filepath = "h5s/"
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath+config["name"]+".hdf5", monitor='val_loss', verbose=1, save_best_only=True,mode='min')
trainModel(mymodel,img,lb,[checkpoint, wandb.keras.WandbCallback()])

Epoch 1/100
Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
282/282 [==============================] - ETA: 0s - loss: 4.1005 - accuracy: 0.0273
Epoch 00001: val_loss improved from inf to 3.65198, saving model to h5s/ResNet_RNN.hdf5
282/282 [==============================] - 40s 141ms/step - loss: 4.1005 - accuracy: 0.0273 - val_loss: 3.6520 - val_accuracy: 0.0273
Epoch 2/100
281/282 [============================>.] - ETA: 0s - loss: 3.6119 - accuracy: 0.0269
Epoch 00002: val_loss improved from 3.65198 to 3.60293, saving model to h5s/ResNet_RNN.hdf5
282/282 [==============================] - 38s 133ms/step - loss: 3.6119 - accuracy: 0.0270 - val_loss: 3.6029 - val_accuracy: 0.0292
Epoch 3/100
281/282 [============================>.] - ETA: 0s - loss: 3.5926 - accuracy: 0.0280
Epoch 00003: val_loss improved from 3.60293 to 3.59954, saving model to h5s/ResNet_RNN.hdf5
282/282 [==============================] - 38s 134ms/step - 

KeyboardInterrupt: 

In [30]:
mymodel.save("./h5s/DenseNet_no_train.h5")